In [73]:
import pandas as pd
import pulp as plp
from pulp import *

class Optimization_Model:
    def __init__(self, name):
        self.name = name
        self.no_of_warehouses = None
        self.no_of_customers = None
        self.set_w = None # set of warehouses
        self.set_c = None # set of customers
        self.cost = {} # dict for cost of arc
        self.lower = {} # dict for lower bounds on arcs
        self.upper = {} # dict for upper bound on arcs
        self.supply = {} # dict for supply at warehouses
        self.demand = {} # dict for demand at customers
        self.x_vars = None # decision variables
        
    def load_parameters(self):
        with open('data_file.txt', "r") as file:
            self.no_of_warehouses = int(file.readline().split()[0])
            self.no_of_customers = int(file.readline().split()[0])
            
            supply = file.readline().split()
            for i, val in enumerate (supply):
                self.supply[i+1] = int(val)
            
            demand = file.readline().split()
            for i, value in enumerate(demand):
                self.demand[i+1] = int(value)
                
            for w in range(self.no_of_warehouses):
                cost = file.readline().split()
                for i, value in enumerate (cost):
                    self.cost[(w+1, i+1)] = int(value)

            for w in range(self.no_of_warehouses):
                upper = file.readline().split()
                for i, value in enumerate (upper):
                    self.upper[(w+1, i+1)] = int(value)

            for w in range(self.no_of_warehouses):
                lower = file.readline().split()
                for i, value in enumerate (lower):
                    self.lower[(w+1, i+1)] = int(value)
        
        self.set_w = range(1, self.no_of_warehouses+1)
        self.set_c = range(1, self.no_of_customers+1)
        
    def run_model(self):
        # initiate LP model object
        self.opt_model = plp.LpProblem(name="Optimization_Model")
        
        #initiate decision vatriables with lower and upper bound
        self.x_vars  = {(i,j): plp.LpVariable(cat=plp.LpInteger,
                                 lowBound = self.lower[i,j], upBound= self.upper[i,j],
                                 name="x_{0}_{1}".format(i,j)) for i in self.set_w for j in self.set_c}
        
        # add demand constraints
        for j in self.set_c:
            self.opt_model.addConstraint(plp.LpConstraint(e=plp.lpSum(self.x_vars[i,j] for i in self.set_w),
                                                          sense=plp.LpConstraintEQ,
                                                          rhs=self.demand[j],
                                                          name="constraint_{0}".format(j)))
            
        #add supply constraints
        for i in self.set_w:
            self.opt_model.addConstraint(plp.LpConstraint(e=plp.lpSum(self.x_vars[i,j] for j in self.set_c),
                                                     sense=plp.LpConstraintLE,
                                                     rhs=self.supply[i],
                                                     name="constraint_{0}".format(i+5)))
        
        #declaring the objective function
        self.objective = plp.lpSum(self.x_vars[i,j] * self.cost[i,j] for i in self.set_w for j in self.set_c)
        
        self.opt_model.sense = plp.LpMinimize
        self.opt_model.setObjective(self.objective)
        self.print_model()
        self.opt_model.solve()
    
    def store_output(self):
        opt_df = pd.DataFrame.from_dict(self.x_vars, orient="index", columns = ["variable_object"])
        opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, names=["warehouse", "customer"])
        opt_df.reset_index(inplace=True)
        opt_df["no of boxes to deliver"] = opt_df["variable_object"].apply(lambda item: item.varValue)
        opt_df.drop(columns=["variable_object"], inplace=True)
        print(opt_df)
        print('\n')
        obj_value = value(model.opt_model.objective)
        print('The toal cost for transportaion is: {}'.format(obj_value))
        
        
    def print_model(self):
        print('Minimize')
        print(self.objective)
        print('\n')
        print('Subject to - ')
        for c in list(model.opt_model.constraints.values()):
            print(c)
        print('\n')

In [74]:
if __name__ == '__main__':
    model = Optimization_Model('model1')
    model.load_parameters()
    model.run_model()
    model.store_output()

Minimize
10*x_1_1 + 12*x_1_2 + 15*x_1_3 + 16*x_1_4 + 20*x_1_5 + 15*x_2_1 + 13*x_2_2 + 15*x_2_3 + 10*x_2_4 + 11*x_2_5


Subject to - 
x_1_1 + x_2_1 = 1500
x_1_2 + x_2_2 = 1600
x_1_3 + x_2_3 = 2500
x_1_4 + x_2_4 = 2200
x_1_5 + x_2_5 = 1200
x_1_1 + x_1_2 + x_1_3 + x_1_4 + x_1_5 <= 4000
x_2_1 + x_2_2 + x_2_3 + x_2_4 + x_2_5 <= 5000


   warehouse  customer  no of boxes to deliver
0          1         1                  1400.0
1          1         2                  1500.0
2          1         3                   800.0
3          1         4                   200.0
4          1         5                   100.0
5          2         1                   100.0
6          2         2                   100.0
7          2         3                  1700.0
8          2         4                  2000.0
9          2         5                  1100.0


The toal cost for transportaion is: 109600.0
